# DATA LOAD

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings
import seaborn as sns

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
travel_24 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/칼럼 추출 데이터셋/24년_1인_관광_.csv')

In [ ]:
travel_24.head()

,ID,SA1_1,SA1_2,SA1_3,SA1_4,SA1_5,MON_EXP_1,MON_EXP_2,MON_EXP_3,MON_EXP_4,...,A9A_11,A9A_12,A10,A11,A12_1,A12_2,A12_3,A12_4,A12_5,A12_6
0,11010680011_420489,1,0,0,0,0,1,2,2,2,...,4.0,4.0,4.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0
1,11010710271_445440,1,0,0,0,0,1,2,2,2,...,4.0,5.0,5.0,4.0,4.0,4.0,5.0,4.0,5.0,4.0
2,11010731041_544424,1,0,0,0,0,1,2,2,2,...,3.0,3.0,4.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0
3,11010731041_544437,1,0,0,0,0,1,2,2,2,...,3.0,3.0,4.0,3.0,5.0,4.0,5.0,4.0,4.0,5.0
4,1102065080A_445449,1,0,0,0,0,1,2,2,2,...,3.0,3.0,4.0,4.0,4.0,4.0,9.0,4.0,4.0,4.0


###필요한 X 칼럼, Y칼럼 정리




### 혼자 여행한 case에 대해서 wide format을 long format으로 변경후 진행

24년 데이터

In [ ]:
phases = [1, 2, 3, 4]
days = list(range(1, 18))

dfs = []

for i in phases:
    for j in days:
        col_name = f'D_TRA{i}_{j}_SPOT'
        if col_name not in travel_24.columns:
           continue
        df_temp = pd.DataFrame()
        df_temp['차수'] = i
        df_temp['여행 인원'] = travel_24.get(f'D_TRA{i}_NUM')
        df_temp['지출 비용'] = travel_24.get(f'D_TRA{i}_COST')
        df_temp['방문 지역'] = travel_24.get(col_name)
        df_temp['여행 일수'] = travel_24.get(f'D_TRA{i}_S_Day')

        df_temp['철도비'] = travel_24.get('A8F_1')
        df_temp['도로비'] = travel_24.get('A8F_4')
        df_temp['식음료비'] = travel_24.get('A8E')
        df_temp['음식점비'] = travel_24.get('A8D')
        df_temp['쇼핑비'] = travel_24.get('A8H')
        df_temp['숙박비'] = travel_24.get('A8C')
        df_temp['교통비'] = travel_24.get('A8F')
        df_temp['여행 활동비'] = travel_24.get('A8G')
        dfs.append(df_temp)

long_df = pd.concat(dfs, ignore_index=True)
long_df = long_df.dropna(subset=['방문 지역'])

long_df['여행 인원'] = long_df['여행 인원'].astype('Int64')
long_df['지출 비용'] = long_df['지출 비용'].astype('Int64')
long_df['방문 지역'] = long_df['방문 지역'].astype('Int64')
long_df['여행 일수'] = long_df['여행 일수'].astype('Int64')

alone24_df = long_df[long_df['여행 인원'] == 1]
alone24_df['방문 지역'] = pd.to_numeric(alone24_df['방문 지역'], errors='coerce')
grouped24_df = alone24_df.groupby('방문 지역').agg({
    # '지출 비용': 'sum',
    '지출 비용': 'mean',
    '철도비': 'mean',
    '도로비': 'mean',
    '식음료비': 'mean',
    '음식점비': 'mean',
    '쇼핑비': 'mean',
    '여행 일수': 'mean',
    '숙박비': 'mean',
    '교통비': 'mean',
    '여행 활동비': 'mean'
}).reset_index()

print(grouped24_df.head())

   방문 지역          지출 비용           철도비           도로비         식음료비  \
0  11010  200468.536082  18348.453608   5065.969072  8886.577320   
1  11020       233187.5  21937.500000   7531.250000  3812.500000   
2  11030       200820.0  13560.000000  15200.000000  3840.000000   
3  11040   99443.181818   3454.545455  14045.454545  4386.363636   
4  11050  130757.142857   7928.571429  11285.714286  5907.142857   

           음식점비           쇼핑비     여행 일수           숙박비           교통비  \
0  75855.639175  24247.422680  0.628866  33814.432990  60703.082474   
1  80062.500000  30156.250000    0.8125  27187.500000  65593.750000   
2  76720.000000  11200.000000      0.64  14400.000000  65880.000000   
3  46045.454545  10409.090909  0.181818  13181.818182  30318.181818   
4  51142.857143  11785.714286       0.5  22857.142857  51842.857143   

         여행 활동비  
0  12164.948454  
1  10593.750000  
2  17040.000000  
3   5454.545455  
4  13357.142857  


In [ ]:
# grouped24_df.to_csv('test.csv', index=False)
# from google.colab import files
# files.download('test.csv')

In [ ]:
car_dep_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/자가용 의존도/car dependency_all.csv')

In [ ]:
car_dep_df.head()

,region_code,car_count,non_car_count,total_count,car_ratio
0,11010,18,83,101,0.178218
1,11020,19,39,58,0.327586
2,11030,11,20,31,0.354839
3,11040,5,13,18,0.277778
4,11050,2,18,20,0.100000


In [ ]:
car_merge_df = car_dep_df[['region_code', 'car_ratio']].copy()

car_merge_df.rename(columns={'region_code': '방문 지역', 'car_ratio': '자가용 의존도'}, inplace=True)

# 병합: 방문 지역 기준으로 인프라 지수 붙이기
merged_df = grouped24_df.merge(car_merge_df, on='방문 지역', how='left')

# 결과 확인
print(merged_df.head())

   방문 지역          지출 비용           철도비           도로비         식음료비  \
0  11010  200468.536082  18348.453608   5065.969072  8886.577320   
1  11020       233187.5  21937.500000   7531.250000  3812.500000   
2  11030       200820.0  13560.000000  15200.000000  3840.000000   
3  11040   99443.181818   3454.545455  14045.454545  4386.363636   
4  11050  130757.142857   7928.571429  11285.714286  5907.142857   

           음식점비           쇼핑비     여행 일수           숙박비           교통비  \
0  75855.639175  24247.422680  0.628866  33814.432990  60703.082474   
1  80062.500000  30156.250000    0.8125  27187.500000  65593.750000   
2  76720.000000  11200.000000      0.64  14400.000000  65880.000000   
3  46045.454545  10409.090909  0.181818  13181.818182  30318.181818   
4  51142.857143  11785.714286       0.5  22857.142857  51842.857143   

         여행 활동비   자가용 의존도  
0  12164.948454  0.178218  
1  10593.750000  0.327586  
2  17040.000000  0.354839  
3   5454.545455  0.277778  
4  13357.142857  0.10000

In [ ]:
# merged_df.to_csv('test.csv', index=False)
# from google.colab import files
# files.download('test.csv')

In [ ]:
infra_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/교통인프라지수_최종(정원)/교통인프라지수_최종 (3).csv')

In [ ]:
infra_df.head()

,Unnamed: 0,시군구코드,교통인프라지수_norm
0,74,31020,3.429257
1,22,11230,3.299027
2,21,11220,2.971521
3,23,11240,2.737020
4,20,11210,2.529984


In [ ]:
infra_merge_df = infra_df[['시군구코드', '교통인프라지수_norm']].copy()

infra_merge_df.rename(columns={'시군구코드': '방문 지역', '교통인프라지수_norm': '교통인프라지수'}, inplace=True)

# 병합: 방문 지역 기준으로 인프라 지수 붙이기
merge_df = merged_df.merge(infra_merge_df, on='방문 지역', how='left')

# 결과 확인
print(merge_df.head())

   방문 지역          지출 비용           철도비           도로비         식음료비  \
0  11010  200468.536082  18348.453608   5065.969072  8886.577320   
1  11020       233187.5  21937.500000   7531.250000  3812.500000   
2  11030       200820.0  13560.000000  15200.000000  3840.000000   
3  11040   99443.181818   3454.545455  14045.454545  4386.363636   
4  11050  130757.142857   7928.571429  11285.714286  5907.142857   

           음식점비           쇼핑비     여행 일수           숙박비           교통비  \
0  75855.639175  24247.422680  0.628866  33814.432990  60703.082474   
1  80062.500000  30156.250000    0.8125  27187.500000  65593.750000   
2  76720.000000  11200.000000      0.64  14400.000000  65880.000000   
3  46045.454545  10409.090909  0.181818  13181.818182  30318.181818   
4  51142.857143  11785.714286       0.5  22857.142857  51842.857143   

         여행 활동비   자가용 의존도   교통인프라지수  
0  12164.948454  0.178218  1.423597  
1  10593.750000  0.327586  1.696777  
2  17040.000000  0.354839  1.328858  
3   5454.545

In [ ]:
# grouped_22_23_df.to_csv('train.csv', index=False)
# from google.colab import files
# files.download('train.csv')

In [ ]:
merge_df.to_csv('test.csv', index=False)
from google.colab import files
files.download('test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>